In [179]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sklearn
from sklearn import metrics, tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [180]:
xls = pd.ExcelFile('Dataset - LBP RA.xlsx')
dataframe = pd.read_excel(xls, 'Training Dataset')

In [181]:
print(dataframe.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546 entries, 0 to 1545
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Treatment                         1546 non-null   int64  
 1   Fever                             1512 non-null   float64
 2   Duration_of_pain                  1515 non-null   float64
 3   Sick_leave                        1546 non-null   int64  
 4   Earlier_hospitalization           1546 non-null   int64  
 5   Workoverload                      459 non-null    float64
 6   Familiy_history                   1546 non-null   int64  
 7   Depression                        1546 non-null   int64  
 8   Extremely_nervous                 1494 non-null   float64
 9   Stress                            1546 non-null   int64  
 10  Relationship_with_colleagues      979 non-null    float64
 11  Irrational_thoughts_risk_lasting  1475 non-null   float64
 12  Irrati

In [182]:
# Mapping integer column
value_columns = ["Decreased_mobility"]
dataframe[value_columns] = dataframe[value_columns].astype("Int64")

# Mapping categories and boolean columns
categorical_columns = ["Treatment", "Weightloss_per_year"]

boolean_columns = [
    "Fever",
    "Sick_leave",
    "Earlier_hospitalization",
    "Workoverload",
    "Familiy_history",
    "Depression",
    "Stress",
    "Uses_analgesics",
    "Uses_corticosteroids",
    "Serious_disease",
    "Neurogenic_signals",
    "Continuous_pain",
    "Nocturnal_pain",
    "Loss_muscle_strength",
    "Trauma",
    "Failure_symptoms",
    "Incoordination",
    "Paidwork",
]

dataframe[categorical_columns] = dataframe[categorical_columns].astype("category")
dataframe[boolean_columns] = dataframe[boolean_columns].astype("boolean")

# Mapping ordinal columns 
age_mapping = {
    "0-19": 0,
    "20-29": 1,
    "30-39": 2,
    "40-49": 3,
    "50-59": 4,
    "60-69": 5,
    "70-79":6,
    ">=80": 7,
}

dataframe["Age"] = dataframe["Age"].replace(age_mapping)

ordinal_columns = [
    "Duration_of_pain",
    "Extremely_nervous",
    "Relationship_with_colleagues",
    "Irrational_thoughts_risk_lasting",
    "Irrational_thoughts_work",
    "Coping_strategy",
    "Kinesiophobia_physical_exercise",
    "Kinesiophobia_pain_stop",
    "Age",
    "neck_pain_intensity",
    "low_back_pain_intensity",
    "arm_left_pain_intensity",
    "arm_right_pain_intensity",
    "leg_left_pain_intensity",
    "leg_right_pain_intensity",
    "working_ability",
]

for column in ordinal_columns:
    dataframe[[column]] = dataframe[[column]].astype("Int64")
    dataframe[column].fillna(-1, inplace=True)
    dataframe[column] = pd.Categorical(dataframe[column], categories=sorted(dataframe[column].unique()), ordered=True)

dataframe

,Treatment,Fever,Duration_of_pain,Sick_leave,Earlier_hospitalization,Workoverload,Familiy_history,Depression,Extremely_nervous,Stress,...,Failure_symptoms,Incoordination,neck_pain_intensity,low_back_pain_intensity,arm_left_pain_intensity,arm_right_pain_intensity,leg_left_pain_intensity,leg_right_pain_intensity,working_ability,Paidwork
0,1,True,10,False,True,<NA>,False,False,0,False,...,True,<NA>,0,8,8,7,6,6,-1,False
1,3,True,10,False,True,False,True,False,0,True,...,True,True,7,8,7,4,0,0,7,True
2,1,True,10,False,True,<NA>,False,False,6,True,...,True,False,3,5,0,2,0,3,-1,False
3,5,True,10,False,True,<NA>,False,False,1,True,...,False,<NA>,2,9,0,0,9,5,-1,False
4,1,True,10,False,True,<NA>,False,False,0,True,...,True,<NA>,0,7,5,0,0,0,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,5,False,8,True,False,<NA>,False,False,3,True,...,True,False,0,8,0,0,0,0,-1,True
1542,1,False,10,False,False,True,False,False,0,True,...,True,<NA>,0,10,0,0,0,9,-1,False
1543,1,False,10,False,True,<NA>,True,True,8,True,...,True,True,0,8,0,0,0,8,-1,False
1544,1,False,10,True,True,<NA>,True,False,3,False,...,True,True,0,8,0,0,0,0,1,True


In [183]:
X = dataframe[[col for col in value_columns + ordinal_columns + categorical_columns if col != "Treatment"]]
y = dataframe['Treatment'] 


# Decision Tree Model
tree_model = DecisionTreeClassifier(max_depth=10)
tree_predicted = cross_val_predict(tree_model, X, y, cv=10)
tree_model.fit(X, y)

# Evaluation
print("Simple k=10 K fold CV")
print("Decision Tree Model:")
print(classification_report(y, tree_predicted))

Simple k=10 K fold CV
Decision Tree Model:
              precision    recall  f1-score   support

           1       0.43      0.50      0.46       659
           2       0.19      0.09      0.13       160
           3       0.00      0.00      0.00        65
           4       0.00      0.00      0.00        13
           5       0.43      0.45      0.44       649

    accuracy                           0.41      1546
   macro avg       0.21      0.21      0.21      1546
weighted avg       0.39      0.41      0.40      1546



In [184]:
class_labels = [str(label) for label in tree_model.classes_]
plt.figure(figsize=(135, 90))
plot_tree(tree_model,max_depth=3, feature_names=X.columns, class_names=class_labels, filled=True, rounded=True)
plt.show()